# Setup

In [1]:
import importlib
import ipywidgets as widgets
import jupyterlab_widgets
import readline
import run_collectors as rc
from IPython.display import display

# Reload helper modules after changes. Mostly used for development.
importlib.reload(rc)

# Do not write history to .python_history to protect credentials
readline.write_history_file = lambda *args: None

In [2]:
# Define available collectors for the user to select later.
selected = rc.define_collectors()
selected = [widgets.Checkbox(value=False,
                             description=c,
                             disabled=False,
                             indent=False) for c in selected]

In [6]:
# Define available hostgroups for the user to collect later
hostgroups = ['nexuses', 'ios_routers', 'ios_switches', 'asas', 'f5_ltms', 'ungrouped']
hostgroups = [widgets.Checkbox(value=False,
                               description=h,
                               disabled=False,
                               indent=False) for h in hostgroups]

# Select and Run Collectors

## Select the hostgroups to run the collectors on

In [8]:
widgets.GridBox(hostgroups, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))

GridBox(children=(Checkbox(value=False, description='nexuses', indent=False), Checkbox(value=False, descriptio…

## Select the collectors to run

In [7]:
widgets.GridBox(selected, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))

GridBox(children=(Checkbox(value=False, description='arp_table', indent=False), Checkbox(value=False, descript…

# Select and Run Validators

# The cells below are a scratchpad for development. They will not be in production.

In [14]:
import ansible_runner
import os
def ansible_get_hostgroup_devices(hostgroup, host_files, quiet=True):
    '''
    Gets the devices inside an Ansible inventory hostgroup.
    Args:
        hostgroup (str):   The Ansible hostgroup
        host_files (list): The path to one or more Ansible host files
                           (I.e., ['inventory/hosts'])
        quiet (bool):      Whether to output the entire graph.
    Returns:
        devices (list):  A list of devices in the hostgroup
    '''
    # TODO: Ansible is slow to read the inventory file. I thought about reading
    #       the YAML files directly, but not all users will be using YAML. So,
    #       there might be another way to optimize this, or it could be
    #       acceptable to leave it as is. I'll have to run more tests.
    graph = ansible_runner.interface.get_inventory('graph',
                                                   host_files,
                                                   quiet=True)
    # TODO: This is a hack. There are better ways to parse the graph.
    graph = str(graph)
    for item in graph.split('@'):
        if hostgroup in item:
            item = item.split(':')[-1]
            item = item.split('|--')[1:-1]
            devices = [i.split('\\')[0] for i in item]
            break
    return devices


def ansible_get_hostgroups(host_files, quiet=True):
    '''
    Gets the devices inside an Ansible inventory hostgroup.
    Args:
        host_files (list): The path to one or more Ansible host files
                           (I.e., ['inventory/hosts'])
        quiet (bool):      Whether to output the entire graph.
    Returns:
        devices (list):  A list of devices in the hostgroup
    '''
    graph = ansible_runner.interface.get_inventory('graph',
                                                   host_files,
                                                   quiet=True)
    graph = str(graph).strip("('")
    # graph = list(filter(None, graph))
    hostgroups = list()
    graph = list(filter(None, graph.split('@')))
    for item in graph:
        hostgroup = item.split(':')[0]
        hostgroups.append(hostgroup)
    return hostgroups


tenant = 'tenant_1'
inventories = [f'~/source/repos/InsightSSG/Net-Runbooks/tenants/{tenant}/inventory/hosts']
inventors = [os.path.expanduser(_) for _ in inventories]
inventories = [os.path.expanduser(f'~/source/repos/InsightSSG/Net-Runbooks/tenants/{tenant}/inventory/hosts')]

hostgroups = ansible_get_hostgroups(inventories)
print(hostgroups)

['all', 'ungrouped']


In [ ]:
# I copied this function from the ansible_runner documentation so that it will
# be easy to reference during development. This function is not part of the
# Net-Manage code.

def get_inventory(action, inventories, response_format=None, host=None, playbook_dir=None,
                  vault_ids=None, vault_password_file=None, output_file=None, export=None, **kwargs):
    '''
    Run an ansible-inventory command to get inventory related details.

    :param action: Valid values are one of ``graph``, ``host``, ``list``
                   ``graph`` create inventory graph, ``host`` returns specific host info and works as inventory script and
                   ``list`` output all hosts info and also works as inventory script.
    :param inventories: List of inventory host path.
    :param response_format: The output format for response. Valid values can be one of ``json``, ``yaml``, ``toml``.
                            Default is ``json``. If ``action`` is ``graph`` only allowed value is ``json``.
    :param host: When ``action`` is set to ``host`` this parameter is used to get the host specific information.
    :param playbook_dir: This parameter is used to sets the relative path for the inventory.
    :param vault_ids: The vault identity to use.
    :param vault_password_file: The vault password files to use.
    :param output_file: The file path in which inventory details should be sent to.
    :param export: The boolean value if set represent in a way that is optimized for export,not as an accurate
                   representation of how Ansible has processed it.
    :param runner_mode: The applicable values are ``pexpect`` and ``subprocess``. Default is set to ``subprocess``.
    :param host_cwd: The host current working directory to be mounted within the container (if enabled) and will be
                     the work directory within container.
    :param envvars: Environment variables to be used when running Ansible. Environment variables will also be
                    read from ``env/envvars`` in ``private_data_dir``
    :param passwords: A dictionary containing password prompt patterns and response values used when processing output from
                      Ansible. Passwords will also be read from ``env/passwords`` in ``private_data_dir``.
    :param settings: A dictionary containing settings values for the ``ansible-runner`` runtime environment. These will also
                     be read from ``env/settings`` in ``private_data_dir``.
    :param ssh_key: The ssh private key passed to ``ssh-agent`` as part of the ansible-playbook run.
    :param quiet: Disable all output
    :param json_mode: Store event data in place of stdout on the console and in the stdout file
    :param artifact_dir: The path to the directory where artifacts should live, this defaults to 'artifacts' under the private data dir
    :param project_dir: The path to the playbook content, this defaults to 'project' within the private data dir
    :param rotate_artifacts: Keep at most n artifact directories, disable with a value of 0 which is the default
    :param timeout: The timeout value in seconds that will be passed to either ``pexpect`` of ``subprocess`` invocation
                    (based on ``runner_mode`` selected) while executing command. It the timeout is triggered it will force cancel the
                    execution.
    :param process_isolation: Enable process isolation, using a container engine (e.g. podman).
    :param process_isolation_executable: Process isolation executable or container engine used to isolate execution. (default: podman)
    :param container_image: Container image to use when running an ansible task (default: quay.io/ansible/ansible-runner:devel)
    :param container_volume_mounts: List of bind mounts in the form 'host_dir:/container_dir:labels. (default: None)
    :param container_options: List of container options to pass to execution engine.
    :param container_workdir: The working directory within the container.
    :param fact_cache: A string that will be used as the name for the subdirectory of the fact cache in artifacts directory.
                       This is only used for 'jsonfile' type fact caches.
    :param fact_cache_type: A string of the type of fact cache to use.  Defaults to 'jsonfile'.
    :param private_data_dir: The directory containing all runner metadata needed to invoke the runner
                             module. Output artifacts will also be stored here for later consumption.
    :param ident: The run identifier for this invocation of Runner. Will be used to create and name
                  the artifact directory holding the results of the invocation.
    :param event_handler: An optional callback that will be invoked any time an event is received by Runner itself, return True to keep the event
    :param cancel_callback: An optional callback that can inform runner to cancel (returning True) or not (returning False)
    :param finished_callback: An optional callback that will be invoked at shutdown after process cleanup.
    :param status_handler: An optional callback that will be invoked any time the status changes (e.g...started, running, failed, successful, timeout)
    :param artifacts_handler: An optional callback that will be invoked at the end of the run to deal with the artifacts from the run.
    :param check_job_event_data: Check if job events data is completely generated. If event data is not completely generated and if
                                 value is set to 'True' it will raise 'AnsibleRunnerException' exception,
                                 if set to 'False' it log a debug message and continue execution. Default value is 'False'
    :type action: str
    :type inventories: list
    :type response_format: str
    :type host: str
    :type playbook_dir: str
    :type vault_ids: str
    :type vault_password_file: str
    :type output_file: str
    :type export: bool
    :type runner_mode: str
    :type host_cwd: str
    :type envvars: dict
    :type passwords: dict
    :type settings: dict
    :type private_data_dir: str
    :type project_dir: str
    :type artifact_dir: str
    :type fact_cache_type: str
    :type fact_cache: str
    :type process_isolation: bool
    :type process_isolation_executable: str
    :type container_image: str
    :type container_volume_mounts: list
    :type container_options: list
    :type container_workdir: str
    :type ident: str
    :type rotate_artifacts: int
    :type timeout: int
    :type ssh_key: str
    :type quiet: bool
    :type json_mode: bool
    :type event_handler: Callable
    :type cancel_callback: Callable
    :type finished_callback: Callable
    :type status_handler: Callable
    :type artifacts_handler: Callable
    :type check_job_event_data: bool

    :returns: Returns a tuple of response and error string. In case if ``runner_mode`` is set to ``pexpect`` the error value is
              empty as ``pexpect`` uses same output descriptor for stdout and stderr. If the vaue of ``response_format`` is ``json``
              it returns a python dictionary object.
    '''

    event_callback_handler = kwargs.pop('event_handler', None)
    status_callback_handler = kwargs.pop('status_handler', None)
    artifacts_handler = kwargs.pop('artifacts_handler', None)
    cancel_callback = kwargs.pop('cancel_callback', None)
    finished_callback = kwargs.pop('finished_callback', None)

    rd = InventoryConfig(**kwargs)
    rd.prepare_inventory_command(action=action, inventories=inventories, response_format=response_format, host=host, playbook_dir=playbook_dir,
                                 vault_ids=vault_ids, vault_password_file=vault_password_file, output_file=output_file, export=export)
    r = Runner(rd,
               event_handler=event_callback_handler,
               status_handler=status_callback_handler,
               artifacts_handler=artifacts_handler,
               cancel_callback=cancel_callback,
               finished_callback=finished_callback)
    r.run()
    response = r.stdout.read()
    error = r.stderr.read()
    if response and response_format == 'json':
        response = json.loads(sanitize_json_response(response))
    return response, error

In [15]:
print(dir(widgets))

['Accordion', 'AppLayout', 'Audio', 'BoundedFloatText', 'BoundedIntText', 'Box', 'Button', 'ButtonStyle', 'CallbackDispatcher', 'Checkbox', 'Color', 'ColorPicker', 'ColorsInput', 'Combobox', 'Controller', 'CoreWidget', 'DOMWidget', 'DatePicker', 'Datetime', 'DatetimePicker', 'Dropdown', 'FileUpload', 'FloatLogSlider', 'FloatProgress', 'FloatRangeSlider', 'FloatSlider', 'FloatText', 'FloatsInput', 'GridBox', 'GridspecLayout', 'HBox', 'HTML', 'HTMLMath', 'Image', 'IntProgress', 'IntRangeSlider', 'IntSlider', 'IntText', 'IntsInput', 'Label', 'Layout', 'NumberFormat', 'Output', 'Password', 'Play', 'RadioButtons', 'Select', 'SelectMultiple', 'SelectionRangeSlider', 'SelectionSlider', 'SliderStyle', 'Stack', 'Style', 'Tab', 'TagsInput', 'Text', 'Textarea', 'TimePicker', 'ToggleButton', 'ToggleButtons', 'ToggleButtonsStyle', 'TwoByTwoLayout', 'TypedTuple', 'VBox', 'Valid', 'ValueWidget', 'Video', 'Widget', '__builtins__', '__cached__', '__doc__', '__file__', '__jupyter_widgets_base_version__'

In [16]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()